# Deep Network for Design Parameters for FWUAV

Importing the Modules

In [1]:
# Modules to import for data preprocessing
import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Modules to import for Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Check for GPU device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Loading the Data

In [3]:
# Loading the data into the pandas dataframe

# Reading the Data
data = pd.read_csv("Design_Data.csv", header=None, dtype = "float32")

print("\n---------------------------------------------------------------------------\n")

# Printing the datafram 
print("\n Data loaded into pandas : \n", data)

print("\n---------------------------------------------------------------------------\n")


scaler = MinMaxScaler()
normal = scaler.fit(data)
normalized_data = normal.transform(data)
normalized_dataframe = pd.DataFrame(normalized_data)
print("\n---------------------------------------------------------------------------\n")

print("\n After Normalisation: \n ", normalized_dataframe)

print("\n---------------------------------------------------------------------------\n")



# Inserting the data columns 
normalized_dataframe.columns = [
    'Flapping Frequency', 'Airspeed', 'Angle Of Attack', 'Normalised Time',
    'Lift', 'Induced Drag', 'Pitching Moment', 'main wing root chord',
    'main wing wingspan', 'main wing tip chord', 'tail backward position',
    'tail root chord', 'tail tip chord', 'tail wingspan'
]

# Split the data into features and targets
X = normalized_dataframe[[
    'Flapping Frequency', 'Airspeed', 'Angle Of Attack', 'Normalised Time',
    'main wing root chord', 'main wing wingspan', 'main wing tip chord',
    'tail backward position', 'tail root chord', 'tail tip chord', 'tail wingspan'
]]
y = normalized_dataframe[['Lift', 'Induced Drag', 'Pitching Moment']]



---------------------------------------------------------------------------


 Data loaded into pandas : 
         0    1     2         3          4         5          6    7    8   \
0      0.1  1.0 -30.0  0.000000  13.921392  1.855566   0.294453  0.3  1.4   
1      0.1  1.0 -30.0  0.500000  -9.041235 -4.439285  22.045515  0.3  1.4   
2      0.1  1.0 -25.0  0.000000  13.925155  1.909448   1.711641  0.3  1.4   
3      0.1  1.0 -25.0  0.500000 -11.013954 -4.369710  20.869232  0.3  1.4   
4      0.1  1.0 -20.0  0.000000  13.789185  1.963994   3.143793  0.3  1.4   
...    ...  ...   ...       ...        ...       ...        ...  ...  ...   
95734  0.5  5.0   5.0  0.915254  -1.671113  0.224382   0.471550  0.3  1.4   
95735  0.5  5.0   5.0  0.932203  -0.796854  0.534728   0.370605  0.3  1.4   
95736  0.5  5.0   5.0  0.949153   0.061979  0.848633   0.258699  0.3  1.4   
95737  0.5  5.0   5.0  0.966102   0.910946  1.153301   0.148894  0.3  1.4   
95738  0.5  5.0   5.0  0.983051   1.726310  1

# Splitting the data into training and testing 

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)


Training features shape: (76591, 11)
Testing features shape: (19148, 11)
Training target shape: (76591, 3)
Testing target shape: (19148, 3)


# Defining the Model

In [5]:
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(11, 128)  # Change input features to 11
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(0.5)  # Dropout layer
        self.fc4 = nn.Linear(32, 3)    # Change output features to 3
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x


In [6]:
# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

In [7]:
print(y_test)

tensor([[0.0140, 0.0110, 0.0095],
        [0.0139, 0.0110, 0.0095],
        [0.0142, 0.0109, 0.0095],
        ...,
        [0.0143, 0.0109, 0.0094],
        [0.0132, 0.0110, 0.0095],
        [0.0141, 0.0111, 0.0095]], device='cuda:0')


# Training and eval of the model 

In [8]:
# Move the model to GPU if available
model = DeepNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping
patience = 100
best_loss = float('inf')
best_model_state = None
no_improvement_epochs = 0

# Training the model
num_epochs = 100
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    train_loss = criterion(outputs, y_train)
    train_loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)

    train_losses.append(train_loss.item())
    test_losses.append(test_loss.item())

    if test_loss < best_loss:
        best_loss = test_loss
        best_model_state = model.state_dict()
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item()}, Test Loss: {test_loss.item()}')

# Load the best model state
if best_model_state is not None:
    model.load_state_dict(best_model_state)


Epoch [10/100], Train Loss: 0.00426141545176506, Test Loss: 0.0006937027210369706
Epoch [20/100], Train Loss: 0.002548813121393323, Test Loss: 0.0015515773557126522
Epoch [30/100], Train Loss: 0.001860624528490007, Test Loss: 0.0009629364358261228
Epoch [40/100], Train Loss: 0.001476559671573341, Test Loss: 0.0008598081767559052
Epoch [50/100], Train Loss: 0.0011804234236478806, Test Loss: 0.0006336360238492489
Epoch [60/100], Train Loss: 0.0009612069115974009, Test Loss: 0.00047903574886731803
Epoch [70/100], Train Loss: 0.0007829685346223414, Test Loss: 0.00039305081008933485
Epoch [80/100], Train Loss: 0.000642208440694958, Test Loss: 0.000298128230497241
Epoch [90/100], Train Loss: 0.0005447787698358297, Test Loss: 0.0002049194445135072
Epoch [100/100], Train Loss: 0.00045982151641510427, Test Loss: 0.00015070920926518738


# testig the data from ptera software


In [9]:
import numpy as np 
import pandas as pd
from Mark4 import simulation, Mk4SaveDataToCSV
from SelfFunctions import extract_second_cycle

In [10]:
import random

def generate_random_decimal(start, end):
    return random.uniform(start, end)

# Example usage
start_value = 1.5
end_value = 5.5
random_number = generate_random_decimal(start_value, end_value)
print("Random number:", random_number)


Random number: 4.0468580043936555


In [11]:
Air_Speed = generate_random_decimal(2, 6)
Angles_of_Attack = generate_random_decimal(-35, 35)
FlappingPeriod = generate_random_decimal(0.5, 1)
Normalisedtime =generate_random_decimal(0,1)
mw_root_chord = generate_random_decimal(0.3, 0.1)
mw_wingspan = generate_random_decimal(0.5, 2)
mw_tip_chord = generate_random_decimal(0.1,0.5)
tail_bposition = generate_random_decimal(0.45, 1.2)
tail_root_chord = generate_random_decimal(0.3,0.7)
tail_tip_chord = generate_random_decimal(0.001,0.1)
tail_wingspan = generate_random_decimal(0.5, 1)

print(f"Air Speed: {Air_Speed}\n"
      f"Angles of Attack: {Angles_of_Attack}\n"
      f"Flapping Period: {FlappingPeriod}\n"
      f"Main Wing Root Chord: {mw_root_chord}\n"
      f"Main Wing Wingspan: {mw_wingspan}\n"
      f"Main Wing Tip Chord: {mw_tip_chord}\n"
      f"Tail Base Position: {tail_bposition}\n"
      f"Tail Root Chord: {tail_root_chord}\n"
      f"Tail Tip Chord: {tail_tip_chord}\n"
      f"Tail Wingspan: {tail_wingspan}")


Air Speed: 3.346288700831801
Angles of Attack: -33.059267229191725
Flapping Period: 0.6057514887855057
Main Wing Root Chord: 0.2589498485729967
Main Wing Wingspan: 1.5741701563507267
Main Wing Tip Chord: 0.18638383231729028
Tail Base Position: 1.1242574385028183
Tail Root Chord: 0.5150925969143321
Tail Tip Chord: 0.02829409779946408
Tail Wingspan: 0.5868776048613806


In [12]:
lift, pitchingmoment, inducedsideforces = simulation(va= Air_Speed, aoa = Angles_of_Attack, fp = FlappingPeriod, mw_airfoil = "naca8304", mw_root_chord = mw_root_chord, mw_wingspan = mw_wingspan, mw_tip_chord = mw_tip_chord, tail_bposition = tail_bposition, tail_type = "V-Tail", tail_root_chord = tail_root_chord , tail_tip_chord = tail_tip_chord, tail_wingspan = tail_wingspan, tail_airfoil = "naca0004" )
print(lift, pitchingmoment, inducedsideforces)

# Flatten the arrays
lift = lift.flatten()
induced_drag = pitchingmoment.flatten()
pitching_moment = inducedsideforces.flatten()
print("Before Second Cycle Extraction :  ", lift.shape, induced_drag.shape, pitching_moment.shape)
print("Before Second Cycle Extraction value:  ", lift, induced_drag, pitching_moment)

# Extract the second cycle from the data
lift, induced_drag, pitching_moment = extract_second_cycle(lift, induced_drag, pitching_moment)
print("After Second Cycle Extraction :  ",lift.shape, induced_drag.shape, pitching_moment.shape)
print("After Second Cycle Extraction value:  ", lift, induced_drag, pitching_moment)

file_name = "test.csv"
Mk4SaveDataToCSV(FlappingPeriod, Air_Speed, Angles_of_Attack, lift, pitching_moment, induced_drag, file_name, 
     mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan)

Simulating:100% |█████████████████████████████████████████████████| Elapsed: 00:31, Remaining: 00:00


[[-23.12288474  -8.53576441   4.13813381   6.96558211   8.9408594
   10.23114129  10.99411899  11.35537038  11.41606204  11.25857328
   10.94966756  10.5424128   10.07765505   9.58544548   9.08655625
    8.59406827   8.11494012   7.65145842   7.20251262   6.76467832
    6.33334737   5.90377181   5.47228057   5.03708627   4.5988161
    4.16056488   3.72758812   3.30651659   2.90458823   2.52871202
    2.18499155   1.87832676   1.61253208   1.39068728   1.21578815
    1.09130103   1.02187423   1.01370689   1.07491363   1.21560357
    1.44771905   1.78420643   2.23808481   2.8208195    3.54037309
    4.3992386    5.39255375   6.50656204   7.71781982   8.99349679
   10.29246301  11.56773909  12.76965944  13.84987385  14.76533858
   15.48207918  15.97797582  16.24428696  16.28572546  16.11913663
   15.77102712  15.27439773  14.66528281  13.97951776  13.25004504
   12.50497666  11.76648407  11.05045213  10.36676672   9.72004599
    9.110608     8.53550118   7.98947965   7.46585404   6.957405

In [28]:


# Input array
input_array = [Air_Speed, Angles_of_Attack, FlappingPeriod, 0.023809523809523808,0,0,0, mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan]
input_array = np.array(input_array)
input_array = input_array.reshape(1, -1)
normalised_input_array = normal.transform(input_array)
indices_to_remove = [4, 5, 6]
normalised_input_array = np.delete(normalised_input_array, indices_to_remove, axis=1)

# Convert to PyTorch tensor
sample_input = torch.tensor(normalised_input_array, dtype=torch.float32)

# Flatten the input tensor to match model's expected input shape
sample_input = sample_input.view(1, -1)

# Send the tensor to the device (CPU or GPU)
sample_input = sample_input.to(device)

# Print the tensor
print("Sample Input Tensor:", sample_input)

# Make predictions
with torch.no_grad():
    predictions = model(sample_input)
    print(predictions)

predictions_numpy = predictions.cpu().numpy()
print(predictions_numpy.reshape(1, -1))
full_prediction_numpy = [Air_Speed, Angles_of_Attack, FlappingPeriod, 0.023809523809523808,predictions_numpy[0][0],predictions_numpy[0][1],predictions_numpy[0][2], mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan]
prediction = normal.inverse_transform(np.array(full_prediction_numpy).reshape(1 , -1))
print(prediction)

Sample Input Tensor: tensor([[ 2.9512, -6.8119,  0.5101,  0.0240,  0.1766,  1.0742, -0.0136,  0.8243,
          1.0335,  0.1365,  0.5410]], device='cuda:0')
tensor([[ 0.0033,  0.0479, -0.0070]], device='cuda:0')
[[ 0.00334244  0.04794657 -0.00702999]]
[[ 3.78091770e+00 -1.64296334e+02  6.34508900e+00  2.36694670e-02
  -1.83862741e+02  5.66529976e+01 -2.01703518e+02  3.33054855e-01
   2.07417016e+00  3.86383835e-01  1.42425731e+00  2.81791659e-01
   6.65881963e-03  6.28189822e-01]]
